In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import numpy as np
from rank_bm25 import BM25Okapi
import pickle as pkl
import pandas as pd
from tqdm.auto import tqdm
import ast


In [ ]:
with open('data/KG_with_ConceptNet_Two_hop.pkl', 'rb') as f:
    IAB_KG=pkl.load(f)

In [ ]:
# with open('data/KG_with_ConceptNet_One_hop.pkl', 'rb') as f:
#     IAB_KG=pkl.load(f)

In [ ]:
with open('data/MAF_KG.pkl', 'rb') as f1:
    MAF_KG=pkl.load(f1)

In [ ]:
head=[]
relation=[]
tail=[]
for items in tqdm(IAB_KG):
    head.append(items[0])
    relation.append(items[1])
    tail.append(items[2])
dic={"IAB Keyword": tail, "Relation": relation, "Object": head}
df_IAB_KG=pd.DataFrame.from_dict(dic)

In [ ]:
head=[]
relation=[]
tail=[]
for items in tqdm(MAF_KG):
    head.append(items[0])
    relation.append(items[1])
    tail.append(items[2])
dic={"MAF Keyword": tail, "Relation": relation, "Object": head}
df_MAF_KG=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [ ]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [ ]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [ ]:
maf=df_MAF['item'].to_list()

In [ ]:
call=iab_labels+maf

In [152]:
expanded_call=[]

for items in tqdm(iab_labels):
    list_items=df_IAB_KG[df_IAB_KG['IAB Keyword']==items]['Object'].tolist()
    string_list=" ".join(list_items)
    string_list=string_list+" "+items
    string_list=string_list.replace("_", " ")

    expanded_call.append(string_list)

 17%|█▋        | 27/160 [00:02<00:09, 13.40it/s]


KeyboardInterrupt: 

In [153]:
#Only for 2D 


expanded_call=[]

for items in tqdm(iab_labels):
    list_items=df_IAB_KG[df_IAB_KG['IAB Keyword']==items]['Object'].tolist()
    string_list=" ".join(list_items)
    string_list=string_list+" "+items
    string_list=string_list.replace("_", " ")
    for second in list_items:
        second_list_items=df_IAB_KG[df_IAB_KG['IAB Keyword']==second]['Object'].tolist()
        second_string_list=" ".join(second_list_items)
        second_string_list=second_string_list+" "+items
        second_string_list=second_string_list.replace("_", " ")

    string_list=string_list+" "+second_string_list

    expanded_call.append(string_list)

100%|██████████| 160/160 [20:47<00:00,  7.80s/it]


In [154]:
for items in tqdm(maf):
    list_items=df_MAF_KG[df_MAF_KG['MAF Keyword']==items]['Object'].tolist()
    string_list=" ".join(list_items)
    string_list=string_list+" "+items
    string_list=string_list.replace("_", " ")

    expanded_call.append(string_list)

100%|██████████| 942/942 [00:08<00:00, 107.70it/s]


In [155]:
with open('data/two_hop_expanded.pkl', 'wb') as f:
    pkl.dump(expanded_call,f)

In [156]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/haskar140/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haskar140/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [157]:
def tokenizer(string, reg="[a-zA-Z'-]+|[0-9]{1,}%|[0-9]{1,}\.[0-9]{1,}%|\d+\.\d+%}"):
    regex=reg
    string=string.replace("-"," ")
    return " ".join(re.findall(regex, string))
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stem the tokens
    porter_stemmer = PorterStemmer()
    tokens = [porter_stemmer.stem(token) for token in tokens]
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    preprocessed_text = tokenizer(preprocessed_text)
    
    return preprocessed_text

In [158]:
iab_expanded=expanded_call[0:160]

In [159]:
maf_expanded=expanded_call[160:]

In [160]:
corpus_clean = [preprocess_text(x) for x in iab_expanded]

In [161]:
tokenized_corpus = [doc.split(" ") for doc in corpus_clean]
print(tokenized_corpus[0])
tokenized_corpus = [list(set(doc)) for doc in tokenized_corpus]
print(tokenized_corpus[0])
bm25 = BM25Okapi(tokenized_corpus)


['hairdress', 'comb', 'haircar', 'care', 'health', 'beauti', 'product', 'shampoo', 'brush', 'hairweav', 'hair', 'care', 'hair', 'care', 'scalp', 'transplant', 'hair', 'bald', 'hair', 'care']
['brush', 'care', 'transplant', 'health', 'beauti', 'hairdress', 'hairweav', 'haircar', 'scalp', 'product', 'bald', 'shampoo', 'hair', 'comb']


In [162]:
def get_bm25_search_hits(query, chunks):
    tokenized_query = preprocess_text(query).split()
    tokenized_query=list(set(tokenized_query))
    sorted=np.sort(bm25.get_scores(tokenized_query), axis=0)[::-1]
    sorted=sorted[sorted>0]
    n=len(sorted)
    print(n)
    if n==0:
        return ["No Pred"]
    top_n = np.argsort(bm25.get_scores(tokenized_query), axis=0)[::-1]
    bm25_search = []
    for idx in top_n:
        if (len(bm25_search) < n):
    #         print(chunks[idx])
    #         print("-----------")
            bm25_search.append(chunks[idx])
        
    return bm25_search

In [163]:
maf_expanded[707]

'genus pug mill wet pugeranian pug pugging pug dog dutch mastiff pugmill prostitute pug nose puck puglike harlot mops dutch bulldog sound pugged pugmark geometrid pugdog dog boxer mini mastiff animal puggish terrier pug nosed deaden mortar footprint puggle tamp chinese pug plug person puggy fill moth chug pugger toy dog mix pugs pugilist stir stop pug'

In [164]:
tokenized_query = preprocess_text(maf_expanded[704]).split()
print(tokenized_query)

['yellow', 'lady', 'slipp']


In [165]:
print(get_bm25_search_hits(maf_expanded[704],iab_labels))

8
['Retail Industry', 'Automotive', 'Remodeling & Construction', 'Shaving', 'Cats', 'Bars & Restaurants', 'Books and Literature', 'Retail Property']


In [166]:
nodes=df_MAF['item'].to_list()
node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [167]:
top_n_labels=[]
for items in tqdm(maf_expanded):
    top_n_labels.append(get_bm25_search_hits(items,iab_labels))

    

  4%|▍         | 36/942 [00:00<00:05, 170.62it/s]

146
16
0
13
118
26
44
126
41
118
82
57
32
74
31
1
60
130
3
7
151
101
19
138
46
73
98
45
113
10
26
8
75
5
106
126
22
86
102
82
120
43


  6%|▌         | 55/942 [00:00<00:05, 176.81it/s]

123
98
13
0
128
58
74
95
123
13
68
75
22
49
28
21
123
89
43
60
23
27
16
46
71
133


  9%|▉         | 88/942 [00:00<00:07, 121.88it/s]

...

8
94
49
83
16
131
0
88
103
10
149
63
93
28
115
18
127
75
146
42
31


 96%|█████████▌| 903/942 [00:06<00:00, 109.77it/s]

147
46
80
129
88
116
118
26
130
42
13
12
144
113
113
49
65
29
7
72
23
71
88


100%|██████████| 942/942 [00:06<00:00, 147.50it/s]

151
108
29
6
100
98
26
114
36
129
6
19
143
50
83
28
44
148
132
27
68
78
29
94
50
5
35
115
23
54


In [168]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [169]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [170]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [171]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    print("Count {}".format(count))
    print(proper)
    print(labels)



    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f

   

    candidate_count+=len(proper)
    for items in labels:
        if items in proper:
            hit_count+=1
    print("C Count {}".format(candidate_count))
    print(hit_count)
    # if count==20:
    #     break


Count 1
['Style & Fashion', "Women's Clothing", "Men's Clothing", "Men's Formal Wear"]
['Sedan', "Women's Handbags and Wallets", 'Houses', 'Healthy Living', 'Hotel Properties', 'Apartments', "Women's Formal Wear", 'Home & Garden', 'Coupe', 'World Cuisines', 'Bars & Restaurants', 'Theater Venues and Events', 'Malls & Shopping Centers', 'Bath and Shower', 'Shopping', 'Deodorant and Antiperspirant', 'Home Appliances', 'Board Games and Puzzles', "Women's Clothing", "Women's Jewelry and Watches", 'Remodeling & Construction', 'Shaving', "Children's Games and Toys", 'SUV', 'Fitness and Exercise', 'Style & Fashion', 'Beach Travel', 'Cats', 'Dogs', 'Casinos & Gambling', "Men's Clothing", 'Books and Literature', 'Games and Puzzles', 'Wellness', 'Television', 'Household Supplies', 'Home Security', 'Movies', "Men's Formal Wear", 'Makeup and Accessories', 'Retail Property', 'Daycare and Pre-School', 'Travel', 'Business and Finance', 'Cooking', 'Home Improvement', 'Education industry', 'Grocery Shop

In [172]:
nodes[19]

'residence'

In [173]:
MAF_predictions[0]

'[\'Style & Fashion\', "Women\'s Clothing", "Men\'s Clothing", "Men\'s Formal Wear"]'

In [ ]:
"Men's Formal Wear" in top_n_labels[0]

In [174]:
hit_count/candidate_count

0.6351249053747161

In [175]:
precision/942

0.04647612451791177

In [176]:
recall/942

0.05588190685543688

In [177]:
f1/942

0.0335769653544577